In [2]:
#7000 wiki is the set of wikipedia articles crawled
#with the ck12 and spelling city.
#the text is not parsed.



import pandas as pd
import os
path = '/Users/MK/GitHub/the_answer_is/data'
os.chdir(path)
train = pd.read_table('training_set_which.tsv',sep = '\t',header = -1)
train.columns = ['id', 'question', 'correctAnswer', 'answerA', 'answerB', 'answerC', 'answerD']
train.head(20)

,id,question,correctAnswer,answerA,answerB,answerC,answerD
0,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...
1,100015,Achromatopsia is a genetic disorder in which s...,B,rod cells,cone cells,lens,iris
2,100024,"The brain, spinal cord, and nerves are organs ...",D,supporting the body and enabling it to move,"transporting oxygen, wastes, and nutrients thr...",producing male and female gametes,conducting messages to coordinate body functions
3,100037,A research team wanted to produce a smaller va...,C,extinction,mutation,selective breeding,natural selection
4,100039,Some areas of the world are experiencing more ...,B,consume large amounts of food,become dormant for long periods,sense infrared and ultraviolet light,transpire water through the skin to the atmosp...
5,100043,With climate and the transfer of energy as hea...,A,"conduction, convection, radiation","reflection, retention, radiation","convection, nuclear, reflection","conduction, solar, nuclear"
6,100051,A family owns a vacation cabin located on a hi...,C,Water is pumped to the cabin from a distant re...,The family brings several gallons of bottled s...,Groundwater is drawn up in buckets from a priv...,The family collects water in rain barrels left...
7,100056,"The plant, Arabidopsis thaliana, has a gene wh...",D,The gene codes for flower color in the plant.,The gene is controlled by environment and not ...,Traits associated with this gene are beneficia...,Traits associated with this gene may vary grea...
8,100064,Trees most likely change the environment in wh...,D,releasing nitrogen in the soil.,crowding out non-native species.,adding carbon dioxide to the atmosphere.,removing water from the soil and returning it ...
9,100066,Blood pressure is often used as an indicator o...,B,cmHg,mmHg,g/cm^3,g/mm^3


In [13]:
import os
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

#train the model using wikipdeida data                
sentences = MySentences('/Users/MK/GitHub/the_answer_is/data/wikipedia_from_all_ck_words_stemmed') # a memory-friendly iterator
model = gensim.models.Word2Vec(sentences,size=300, window=5, min_count=1, workers=4)
#procedure needed for deleting words not in the training set. 
def only_the_words_in_index( list, index ):
    output = []
    for a, s in enumerate(list):
        if s in index:
            output.append(list[a])
    return output

def string_stemmer(line):
    import re
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    stemmer = SnowballStemmer("english")
    line = re.sub(r'[^a-zA-Z ]',r'',line)
    line = line.split()
    line = [word for word in line if word not in stopwords.words('english')]  # remove the stop words. 
    output = []
    for word in line:
        output.append(stemmer.stem(word))     #stem all words 
    output = ' '.join(output)           # join the list to make a string
    return output

In [19]:
import numpy as np 
model = model
myanswer_list = []
myanswer_distance = pd.DataFrame(np.zeros(4).reshape(1,4), columns = ['A','B','C','D'])    #initialize dataframe to store my answers distance


for i in train.index.values:
    length = len(train)
    q = train.ix[i][1]
    a1 = train.ix[i][3]
    a2 = train.ix[i][4]
    a3 = train.ix[i][5]
    a4 = train.ix[i][6]
    
    q = string_stemmer(q)
    a1 = string_stemmer(a1)
    a2 = string_stemmer(a2)
    a3 = string_stemmer(a3)
    a4 = string_stemmer(a4)
    
    q  = q.split()
    a1 = a1.split()
    a2 = a2.split()
    a3 = a3.split()
    a4 = a4.split()
    
    q =  only_the_words_in_index( q, model.index2word)
    a1 = only_the_words_in_index( a1, model.index2word)
    a2 = only_the_words_in_index( a2, model.index2word)
    a3 = only_the_words_in_index( a3, model.index2word)
    a4 = only_the_words_in_index( a4, model.index2word)

    try:
        answer_similarity = np.array([['A',model.n_similarity(q, a1)],['B',model.n_similarity(q, a2)],
                                      ['C',model.n_similarity(q, a3)],['D',model.n_similarity(q, a4)]])
    except:
        print 'Error on ', i, ' and set lengths at random'
        answer_similarity = np.array([['A',np.random.rand()],['B',np.random.rand()],
                                      ['C',np.random.rand()],['D',np.random.rand()]])
        
        myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
        myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
        myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
        myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )
        
        #myanswer_index = answer_similarity[:,1].argsort()[-2]        #get the second best 
        myanswer_index = answer_similarity[:,1].argmax()         #get the maximum similarity 
        myanswer = answer_similarity[myanswer_index][0]
        myanswer_list.append(myanswer)
        continue
    
    myanswer_distance.set_value(i, 'A', answer_similarity[0,1] )    #write down distance for each choice
    myanswer_distance.set_value(i, 'B', answer_similarity[1,1] )
    myanswer_distance.set_value(i, 'C', answer_similarity[2,1] )
    myanswer_distance.set_value(i, 'D', answer_similarity[3,1] )
    
    #myanswer_index = answer_similarity[:,1].argsort()[-2]
    myanswer_index = answer_similarity[:,1].argmax()          #get the maximum similarity 
    myanswer = answer_similarity[myanswer_index][0]
    myanswer_list.append(myanswer)
    
    print 'progress: ', i, '/', length 
#for printing out the distance
# myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_distance.csv', encoding='utf-8')    
train['ck_12_word2vec_answer'] = myanswer_list
    

progress:  0 / 308
progress:  1 / 308
progress:  2 / 308
progress:  3 / 308
progress:  4 / 308
progress:  5 / 308
progress:  6 / 308
progress:  7 / 308
progress:  8 / 308
Error on  9  and set lengths at random
progress:  10 / 308
progress:  11 / 308
progress:  12 / 308
progress:  13 / 308
progress:  14 / 308
progress:  15 / 308
progress:  16 / 308
progress:  17 / 308
progress:  18 / 308
progress:  19 / 308
progress:  20 / 308
progress:  21 / 308
progress:  22 / 308
progress:  23 / 308
progress:  24 / 308
progress:  25 / 308
progress:  26 / 308
progress:  27 / 308
progress:  28 / 308
progress:  29 / 308
progress:  30 / 308
progress:  31 / 308
progress:  32 / 308
progress:  33 / 308
progress:  34 / 308
progress:  35 / 308
progress:  36 / 308
progress:  37 / 308
progress:  38 / 308
progress:  39 / 308
progress:  40 / 308
progress:  41 / 308
progress:  42 / 308
progress:  43 / 308
progress:  44 / 308
progress:  45 / 308
progress:  46 / 308
progress:  47 / 308
progress:  48 / 308
progress: 

/Users/MK/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [20]:
train

,id,question,correctAnswer,answerA,answerB,answerC,answerD,ck_12_word2vec_answer
0,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...,B
1,100015,Achromatopsia is a genetic disorder in which s...,B,rod cells,cone cells,lens,iris,C
2,100024,"The brain, spinal cord, and nerves are organs ...",D,supporting the body and enabling it to move,"transporting oxygen, wastes, and nutrients thr...",producing male and female gametes,conducting messages to coordinate body functions,D
3,100037,A research team wanted to produce a smaller va...,C,extinction,mutation,selective breeding,natural selection,C
4,100039,Some areas of the world are experiencing more ...,B,consume large amounts of food,become dormant for long periods,sense infrared and ultraviolet light,transpire water through the skin to the atmosp...,B
5,100043,With climate and the transfer of energy as hea...,A,"conduction, convection, radiation","reflection, retention, radiation","convection, nuclear, reflection","conduction, solar, nuclear",A
6,100051,A family owns a vacation cabin located on a hi...,C,Water is pumped to the cabin from a distant re...,The family brings several gallons of bottled s...,Groundwater is drawn up in buckets from a priv...,The family collects water in rain barrels left...,A
7,100056,"The plant, Arabidopsis thaliana, has a gene wh...",D,The gene codes for flower color in the plant.,The gene is controlled by environment and not ...,Traits associated with this gene are beneficia...,Traits associated with this gene may vary grea...,C
8,100064,Trees most likely change the environment in wh...,D,releasing nitrogen in the soil.,crowding out non-native species.,adding carbon dioxide to the atmosphere.,removing water from the soil and returning it ...,D
9,100066,Blood pressure is often used as an indicator o...,B,cmHg,mmHg,g/cm^3,g/mm^3,B


In [21]:
train['ck_12_word2vec_correct'] = (train['correctAnswer'] == train['ck_12_word2vec_answer'])

In [22]:
print 'percent correct is ' , train['ck_12_word2vec_correct'].sum(axis =0) / (len(train) + 0.0)

percent correct is  0.409090909091


Observation1:
1. If you stem both the training documents and the questions,
the percent increased from 0.29 to 0.34.

In [32]:
train.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_7000wiki_stemmed.csv', encoding='utf-8')
myanswer_distance.to_csv('/Users/MK/GitHub/the_answer_is/data/answer/pure_ck12_word2vec_7000wiki_stemmed_distance.csv', encoding='utf-8')  

In [33]:
myanswer_distance

,A,B,C,D
0,0.40773,0.449015,0.364753,0.351798
1,0.132015948959,0.167173,0.158588,0.045031
2,0.143791958045,0.320548,0.366202,0.430643
3,0.413309092597,0.150744,0.313763,0.459602
4,0.927427972846,0.931573,0.860712,0.880432
5,0.291560584901,0.303703,0.268605,0.352822
6,0.193136990345,0.200972,0.270078,0.340348
7,0.182266921373,0.250053,0.108819,0.319832
8,0.173724123041,0.121427,0.055383,0.141560
9,0.565458652792,0.557725,0.579249,0.568656
